# Обработка большого текста

Текстовые файлы, например PDF, зачастую содержат тексты, размеры которых превышают размер конекста модели.
Для обработки таких текстов вы можете попробовать следующие подходы:

* Смените модель. Попробуйте использовать модель с большим контекстом, например, [GigaChat Lite+](https://developers.sber.ru/docs/ru/gigachat/models).
* Разделите документ на небольшие фрагменты и попробуйте извлечь данные из них.
* Используйте RAG — разделите документ на фрагменты и проиндексируйте их. После этого можно будет извлекать данные только из тех фрагментов, которые кажутся модели подходящими.

Каждый из способов имеет свои плюсы и минусы, и подходит для решения различных задач.

В этом разделе вы найдете примеры реализации второго и третьего подходов.

## Подготовка

В качестве примера используется [статью о машинах из Википедии](https://en.wikipedia.org/wiki/Car), загруженная как документ (`Document`) GigaChain.

In [2]:
import re

import requests
from langchain_community.document_loaders import BSHTMLLoader

# Загрузка статьи
response = requests.get("https://en.wikipedia.org/wiki/Car")
# Запись в файл
with open("car.html", "w", encoding="utf-8") as f:
    f.write(response.text)
# Загрузка файла с помощью парсера HTML
loader = BSHTMLLoader("car.html")
document = loader.load()[0]
# Очистка кода
# Замена нескольких последовательных новых строк одной новой строкой
document.page_content = re.sub("\n\n+", "\n", document.page_content).replace(
    "\xa0", " "
)

In [3]:
print(len(document.page_content))

79251


## Определение схемы данных

Используем Pydantic для определения схемы данных, которые нужно извлечь с помощью модели.
В приведенном примере извлечем список «основных этапов» (например, важных исторических событий), которые включают год и описание.

В примере также задается поле `evidence`, а модели поручается предоставить дословные цитаты из статьи, которые подтверждают извлеченные данные.
Это позволяет нам сравнить результаты извлечения с текстом из оригинального документа.

In [4]:
from typing import List, Optional

from langchain_community.chat_models.gigachat import GigaChat
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field


class KeyDevelopment(BaseModel):
    """Важная историческая дата."""

    # Док-строка выше, передается в описании функции
    # и помогает улучшить результаты работы LLM

    # Обратите внимание:
    # У каждого поля есть описание (`description`), которое передается в модель, в описании аргументов функции.
    # Хорошее пописание помогает повысить качество извлечения.
    year: Optional[int] = Field(
        ..., description="Год исторического события. Не может быть null."
    )
    description: str = Field(
        ..., description="Описание. Что произошло в этом году? Каково было развитие?"
    )
    evidence: str = Field(
        ...,
        description="Повтори дословно предложения из текста, из которых были извлечены год и описание.",
    )


class ExtractionData(BaseModel):
    """Извлеченая информация о ключевых событиях в истории."""

    key_developments: List[KeyDevelopment]


# Определяем промпт: добавляем инструкции и дополнительный контекст
# На этом этапе можно:
# * Добавить примеры работы функций, для улучшения качества извлечения информации
# * Предоставить дополнительную информацию о том какие данные и откуда будут извлекаться
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Ты эксперт в извлечении важных исторических дат из текста. "
            "Извлекай только важные исторические события с годами."
            "Если ты не можешь извлечь год, не записывай это в историческое событие",
        ),
        MessagesPlaceholder(
            "examples"
        ),  # В блоках ниже показано как использовать примеры для повышения качества извлечения данных
        ("human", "{text}"),
    ]
)

/Users/mikelarg/Projects/gigachain/libs/core/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `with_structured_output` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


## Создание экстрактора

Пример экстрактора, созданного с помощью GigaChat-Pro.

In [5]:
from langchain_community.chat_models.gigachat import GigaChat

llm = GigaChat(
    verify_ssl_certs=False,
    timeout=6000,
    model="GigaChat-Pro",
    temperature=0.01,
)

In [6]:
extractor = prompt | llm.with_structured_output(
    schema=ExtractionData,
)

## Разделение файла на фрагменты

Разделение файла на фрагменты, которые помещаются в окно контекста модели.

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Управление размером каждого фрагмента
    chunk_size=2000,
    # Управление перекрытием между фрагментами
    chunk_overlap=20,
)

texts = text_splitter.split_text(document.page_content)

Добавляем образцы, на которые будет ориентироваться модель при генерации данных.

In [6]:
import itertools
from typing import List, TypedDict

from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    FunctionMessage,
    HumanMessage,
)
from langchain_core.pydantic_v1 import BaseModel

#
# В этом блоке добавляются примеры работы функций для повышения качества извлечения данных
# Подробнее о примерах работы — в example.ipynb
#


class Example(TypedDict):
    """Пример работы функций."""

    input: str  # Пример вызова
    function_calls: List[BaseModel]  # Pydantic-модель с примером извлечения
    function_outputs: List[str]


def tool_example_to_messages(example: Example) -> List[BaseMessage]:
    """Превращаем примеры вызовов функций в историю сообщений"""
    messages: List[BaseMessage] = [HumanMessage(content=example["input"])]
    for function_call, function_output in itertools.zip_longest(
        example["function_calls"], example.get("function_outputs", [])
    ):
        messages.append(
            AIMessage(
                content="",
                additional_kwargs={
                    "function_call": {
                        # Сейчас название модели соответствует pydantic-модели
                        # В текущий момент в API это неочевидно и будет улучшено.
                        "name": function_call.__class__.__name__,
                        "arguments": function_call.dict(),
                    },
                },
            )
        )
        output = "You have correctly called this tool."
        if function_output:
            output = function_output
        messages.append(
            FunctionMessage(name=function_call.__class__.__name__, content=output)
        )
    return messages


examples = [
    (
        "Техногенная авария «Размыв» Ленинградского-Петербургского метрополитена "
        "является крупнейшей в мировой практике метростроения[34]; была экранизирована "
        "в фильме «Прорыв» и послужила вдохновением для фильма «Метро»[35].",
        ExtractionData(
            key_developments=[
                KeyDevelopment(
                    year=None,
                    description="Техногенная авария 'Размыв' в "
                    "Ленинградском-Петербургском метрополитене является "
                    "крупнейшей в мировой практике метростроения",
                    evidence="была экранизирована в фильме 'Прорыв' и послужила "
                    "вдохновением для фильма 'Метро'",
                )
            ]
        ),
        """pydantic.v1.error_wrappers.ValidationError: 1 validation error for KeyDevelopment
year
  none is not an allowed value (type=type_error.none.not_allowed)""",
    ),
    (
        "In 1891, Auguste Doriot and his Peugeot colleague Louis Rigoulot completed "
        "the longest trip by a petrol-driven vehicle when their self-designed and "
        "built Daimler powered Peugeot Type 3 completed 2,100 kilometres (1,300 mi) "
        "from Valentigney to Paris and Brest and back again. They were attached to "
        "the first Paris–Brest–Paris bicycle race, but finished six days "
        "after the winning cyclist, Charles Terront.",
        ExtractionData(
            key_developments=[
                KeyDevelopment(
                    year=1891,
                    description="Август Дорио и его коллега Луи Риголу "
                    "завершают самую длинную поездку на бензиновом автомобиле",
                    evidence="In 1891, Auguste Doriot and his Peugeot colleague Louis Rigoulot completed the longest trip by a petrol-driven vehicle",
                )
            ]
        ),
        "You have correctly called this tool.",
    ),
    (
        "I love cats and dogs.",
        ExtractionData(key_developments=[]),
        "You have correctly called this tool.",
    ),
]


messages = []

for text, tool_call, function_output in examples:
    messages.extend(
        tool_example_to_messages(
            {
                "input": text,
                "function_calls": [tool_call],
                "function_outputs": [function_output],
            }
        )
    )

Используйте метод `.batch`, для параллельного извлечения данных из каждого фрагмента.

<!--
:::note

You can often use .batch() to parallelize the extractions! `batch` uses a threadpool under the hood to help you parallelize workloads.

If your model is exposed via an API, this will likley speed up your extraction flow!

:::
-->

In [ ]:
# Добавьте ограничение на работу с первыми тремя фрагментами
# чтобы можно было быстро перезапускать код
first_few = texts[:10]

extractions = extractor.batch(
    [{"text": text, "examples": messages} for text in first_few],
    {
        "max_concurrency": 5
    },  # ограничьте конкурентность с помощью параметра max_concurrency
)

### Объединение результатов

После извлечения данных из разных фрагментов их нужно объединить в общий результат.

In [11]:
key_developments = []

for extraction in extractions:
    key_developments.extend(extraction.key_developments)

key_developments[:10]

[KeyDevelopment(year=None, description='Car, or an automobile, is a motor vehicle with wheels.', evidence='A car, or an automobile, is a motor vehicle with wheels.'),
 KeyDevelopment(year=1769, description='Французский изобретатель Николя-Жозеф Кюньо создал первый паровой автомобиль в 1769 году', evidence='French inventor Nicolas-Joseph Cugnot built the first steam-powered road vehicle in 1769'),
 KeyDevelopment(year=1886, description='Немецкий изобретатель Карл Бенц запатентовал свой Benz Patent-Motorwagen в 1886 году', evidence='The modern car—a practical, marketable automobile for everyday use—was invented in 1886, when German inventor Carl Benz patented his Benz Patent-Motorwagen.'),
 KeyDevelopment(year=1908, description='Модель Т, американский автомобиль, произведенный компанией Ford Motor Company, стал доступным для масс в 1908 году', evidence='One of the first cars affordable by the masses was the 1908 Model T, an American car manufactured by the Ford Motor Company.'),
 KeyDeve

## Использование методики RAG

Вы также можете проиндексировать фрагменты текста и извлекать данные только из наиболее подходящих из них.

:::caution

При определении подходящих фрагментов могут возникнуть проблемы.

Так, при работе с представленной статьей из Википедии этот подход может привести к потере многих данных, так как фрагменты будут опознаны как не подходящие.

Попробуйте использовать подход для решения своих задач, чтобы понять подходит он вам или нет.

:::

Для реализации подхода, основанного на RAG: 

1. Разделите файлы на фрагменты и проиндексируйте их (например, с использованием векторного хранилища);
2. Добавьте извлечение данных из векторного хранилища перед вызовом цепочки `extractor`.

Простой пример, использующий векторное хранилище `FAISS`.

In [40]:
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda, RunnableParallel
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

texts = text_splitter.split_text(document.page_content)
vectorstore = FAISS.from_texts(texts, embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever(
    search_kwargs={"k": 3}
)  # Извлечение данных только из этого документа

В приведенном примере RAG-экстрактор извлекает данные только из начала документа.

In [41]:
def combine_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])


rag_extractor = {
    "text": retriever | combine_docs,  # получение содержимого начала документа
    "examples": lambda x: messages,
} | extractor

In [42]:
results = rag_extractor.invoke("Key developments")

Giga generation stopped with reason: function_call


In [43]:
for key_development in results.key_developments:
    print(key_development)

year=2018 description='Рост популярности автомобилей и поездок привел к заторам на дорогах.' evidence='Так, Москва, Стамбул, Богота, Мехико и Сан-Паулу были самыми загруженными городами в 2018 году, согласно данным компании INRIX, специализирующейся на анализе данных.'
year=1924 description='В Европе происходило то же самое.' evidence='Morris начал производство на конвейере в Ковли в 1924 году и вскоре стал продавать больше автомобилей, чем Ford, а также начал следовать практике вертикальной интеграции Ford, покупая двигатели, коробки передач и радиаторы у других компаний.'
year=None description='В Японии производство автомобилей было ограничено до Второй мировой войны.' evidence='Только несколько компаний производили автомобили в ограниченном количестве, и эти автомобили были небольшими, трехколесными для коммерческих целей или были результатом партнерства с европейскими компаниями.'
year=None description='Большинство автомобилей, используемых в начале 2020-х годов, работают на бензин

## Известные проблемы

При реализации каждого из подходов вы можете столкнуться со следующими проблемами:

* При делении текста на фрагметы модель возможно не сможет извлечь нужные данные, если они встречаются в разных фрагментах.
* Большое перекрытие между фрагментами может привести к задвоению информации.
* Модели могут придумывать данные.